#  Individual Project

<img src="fer.png" width=100 height=100 align="right">

Author: Fernando Godínez Cornejo

Contact: fernando.godinez@student.ie.edu

Web: linkedin.com/fernandogodinezc

Last revision: 22/Aug/2021

# Download libraries to be used

In [1]:
!pip install --upgrade git+http://github.com/renero/dataset
!pip install skrebate
!pip install gplearn

from copy import copy
from dataset import Dataset
from datetime import datetime, date
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

import re
import math
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

  Cloning http://github.com/renero/dataset to /private/var/folders/wv/yvq0c5md1zj9wnt_vwjs6vpc0000gn/T/pip-req-build-2t4jrf99


  Created wheel for dataset: filename=dataset-0.17.1-py3-none-any.whl size=22736 sha256=929a315dc6fd8596c7b411fbf27a2b208eed875558a21dbe414c1588d1b39e35
  Stored in directory: /private/var/folders/wv/yvq0c5md1zj9wnt_vwjs6vpc0000gn/T/pip-ephem-wheel-cache-ruoyxddp/wheels/5e/d1/b3/9ef477d13c9620bcfd5984012b4831ba1a2641562d7b0e6b78
Successfully built dataset
  Attempting uninstall: dataset
    Found existing installation: dataset 0.17.1
    Uninstalling dataset-0.17.1:
      Successfully uninstalled dataset-0.17.1


# Class creation

Hi Professor!

My idea is to develop a flow for the end user to handle any type of document by just providing the inputs (columns).

The class has four main sections:
1. Initialization: to upload the dataframe from any csv
2. Cleaning: Composed by six different functions (private) and ordered in a logic flow. 
3. Categorical Analysis 
4. Numerical Analysis

In [2]:
class RiskEvaluation():

    # Class initialization by giving the dataFrame
    def  __init__(self,filename) :
        self.data = pd.read_csv(filename)
        
    # Cleaning of data
    def inicial (self,piv,birth_date,target,down_payment,income_status,dates_todays) :
        self._pivot_unique(piv)
        self._clean_target(target)
        self._clean(birth_date)
        self._down(down_payment)
        self._income(income_status)
        self._dayslapsed(dates_todays)
        return self.data
    
    #Remove duplicates using the 'pivot' value established by the user
    def _pivot_unique (self,piv) :
        self.data.drop_duplicates(subset=[piv], keep='last', inplace = True)
        return self.data
    
    #The idea of this analysis is to have a binary bucket 
    def _clean_target (self,target) :
        for i in range(len(self.data.columns)):
            tar = str(self.data.columns[i])
            if tar == target :
                val = np.where(self.data[self.data.columns[i]]>0,1,self.data[self.data.columns[i]])
                self.data[self.data.columns[i]] = val
        return self.data
    
    # Getting the agre from the birth date in the dataFrame and cleainig empty spaces
    def _clean(self,birth_date) :

        data = Dataset.from_dataframe(self.data)
        numerical_features = data.numerical_features
        categorical_features = data.categorical_features
        
        # clean numerical values
        for i in range(len(self.data.columns)):
            empty = self.data[self.data.columns[i]].isna().any()
            if empty == True and self.data.columns[i] in numerical_features:
                val = self.data[self.data.columns[i]].fillna(self.data[self.data.columns[i]].mean())
                self.data[self.data.columns[i]] = val
        
        # clean categorical values
            # fill empty
        for i in range(len(self.data.columns)):
            empty = self.data[self.data.columns[i]].isna().any()
            if empty == True and self.data.columns[i] in categorical_features:
                val = self.data[self.data.columns[i]].fillna('UNKNOWN')
                self.data[self.data.columns[i]] = val
            
            # upper case
        for i in range(len(self.data.columns)):
            if self.data.columns[i] in categorical_features:
                val = self.data[self.data.columns[i]].str.upper()
                self.data[self.data.columns[i]] = val
        
        # if birth date, return age
        def age(born):
            while True:
                try:
                    born = datetime.datetime.strptime(born, "%Y-%m-%d").date()
                    today = date.today()
                    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
                except ValueError:
                    return 'UNKNOWN'
        for i in range(len(self.data.columns)):
            dateb = str(self.data.columns[i])
            if self.data.columns[i] in categorical_features and dateb == birth_date :
                val = self.data[self.data.columns[i]].apply(age)
                self.data[self.data.columns[i]] = val

        return self.data

    # The down Payment has to mean something in order to be useful in the Model. We get the % in each value. Also we separate between individuals and corporate
    
    def _down(self,down_payment) :
        def pay(payment) :
            y = re.findall('\d+', payment)
            if len(y) > 0 :
                result = int(y[0])/100
            else:
                result = 0
            return result
        
        def ty(types) :
            if 'EMPLOYED' in types :
                return 'EMPLOYED'
            else:
                return 'CORPORATE'
        
        self.data["DOWN_PAYMENT"] = None
        self.data["TYPE"] = None
        val = self.data[down_payment].apply(pay)
        self.data["DOWN_PAYMENT"] = val
        
        val = self.data[down_payment].apply(ty)
        self.data["TYPE"] = val
        self.data.drop(columns = [down_payment], inplace = True)
        
        return self.data
    
    # There are too many jobs, we can isolate between those who get some income and unemployed guys
    def _income (self,income_status) :
        def income(incomes) :
            if 'UNEMPLOYED' in incomes :
                return 'UNEMPLOYED'
            else:
                return 'ACTIVE' 
       
        val = self.data[income_status].apply(income)
        self.data[income_status] = val
    
    # Dates are not useful for the model, we need numerical values.
    
    def _dayslapsed (self,dates_todays):
        def daily(dai) :
            change = datetime.datetime.strptime(dai, "%Y-%m-%d").date()
            today = date.today()
            delta = today - change
            return delta.days
        
        for i in range(len(dates_todays)):
            self.data[dates_todays[i]+'_DAYS_LAPSED'] = None
            val = self.data[dates_todays[i]].apply(daily)
            self.data[dates_todays[i]+'_DAYS_LAPSED'] = val
            self.data.drop(columns = [dates_todays[i]], inplace = True)
    
    
    # Lets use categorical values established by the user
    
    def set_train_cat (self,target_value,seg_data):
        
        df_random_sample, _ = train_test_split(self.data, test_size=0.80)
        
        
        def get_specific_columns(df_random_sample, data_types, to_ignore = list(), ignore_target = False):
            columns = df_random_sample.select_dtypes(include=data_types).columns
            if ignore_target:
                columns = filter(lambda x: x not in to_ignore, list(columns))
            return list(columns)
        
        all_numeric_variables = get_specific_columns(df_random_sample, ["float64", "int64"], [target_value], ignore_target = True)
        
        splitter = train_test_split
        df_train, df_test = splitter(df_random_sample, test_size = 0.2, random_state = 42)
        
        X_train = df_train[all_numeric_variables]
        y_train = df_train[target_value]
        
        X_test = df_test[all_numeric_variables]
        y_test = df_test[target_value]
        
        method = LogisticRegression(random_state=0)
        fitted_full_model = method.fit(X_train, y_train)
        y_pred = fitted_full_model.predict(X_test)
        
        # Result accuracy all model
        full_model = accuracy_score(y_test, y_pred)
        
        result_full_model_etal =  ["The total accuracy using all variable is: " + str(full_model)]
        
        #Analysis Model
        for seg in range(len(seg_data)):
            max_value_seg = self.data[seg_data[seg]].value_counts().idxmax()
            
            # set dataframes of train and test
            
            df_train_seg1 = df_train[df_random_sample[seg_data[seg]] == max_value_seg]
            df_train_seg2 = df_train[df_random_sample[seg_data[seg]] != max_value_seg]
            df_test_seg1 = df_test[df_random_sample[seg_data[seg]] == max_value_seg]
            df_test_seg2 = df_test[df_random_sample[seg_data[seg]] != max_value_seg]
            
            #getting results seg 1 vs seg 1
           
            X_train_seg1 = df_train_seg1[all_numeric_variables]
            y_train_seg1 = df_train_seg1[target_value]
            X_test_seg1 = df_test_seg1[all_numeric_variables]
            y_test_seg1 = df_test_seg1[target_value]
            fitted_model_seg1 = method.fit(X_train_seg1, y_train_seg1)
            y_pred_seg1 = fitted_model_seg1.predict(X_test_seg1)
            y_pred_seg1_fullmodel = fitted_full_model.predict(X_test_seg1)
            
            result_full_model_etal.append("Using: " + seg_data[seg] + " Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):"+str(accuracy_score(y_test_seg1, y_pred_seg1)))
            result_full_model_etal.append("Using: " + seg_data[seg] + " Model Developed on Full Population (train sample) applied on Seg 1 (test sample):"+str(accuracy_score(y_test_seg1, y_pred_seg1_fullmodel)))
            
            #getting results seg 2 vs seg 2
            
            X_train_seg2 = df_train_seg2[all_numeric_variables]
            y_train_seg2 = df_train_seg2[target_value]
            X_test_seg2 = df_test_seg2[all_numeric_variables]
            y_test_seg2 = df_test_seg2[target_value]
            fitted_model_seg2 = method.fit(X_train_seg2, y_train_seg2)
            y_pred_seg2 = fitted_model_seg2.predict(X_test_seg2)
            y_pred_seg2_fullmodel = fitted_full_model.predict(X_test_seg2)
            result_full_model_etal.append("Using: " + seg_data[seg] +" Model Developed on Full Population (train sample) applied on Seg 2 (test sample):" + str(accuracy_score(y_test_seg2, y_pred_seg2_fullmodel)))
            result_full_model_etal.append("Using: " + seg_data[seg] +" Model Developed on Seg 2 (train sample) applied on Seg 2 (test sample):" + str(accuracy_score(y_test_seg2, y_pred_seg2))) 
        
        return result_full_model_etal
    
    
    
    # Now numercial columns
    
    def set_train_num (self,seg_data_cat,target_value,seg_data_num): 
        
        #One hot encoding
        data = Dataset.from_dataframe(self.data)
        
        for seg in range(len(seg_data_cat)):
            data.onehot_encode(seg_data_cat[seg])
            data.drop_columns(seg_data_cat[seg])
        
        self.data = data.features
        
        # Lets get rid of Unknown values so that we can have means in each column
        for dro in range(len(seg_data_num)):
            self.data.drop(self.data.index[self.data[seg_data_num[dro]] == 'UNKNOWN'], inplace = True)
        
        def get_specific_columns(df_random_sample, data_types, to_ignore = list(), ignore_target = False):
            columns = df_random_sample.select_dtypes(include=data_types).columns
            if ignore_target:
                columns = filter(lambda x: x not in to_ignore, list(columns))
            return list(columns)
        
        #select data
        df_random_sample, _ = train_test_split(self.data, test_size=0.80)
        all_numeric_variables = get_specific_columns(df_random_sample, ["float64", "int64"], [target_value], ignore_target = True)
        
        splitter = train_test_split
        df_train, df_test = splitter(df_random_sample, test_size = 0.2, random_state = 42)
        
        result_full_model_etal = []
        
        X_train = df_train[all_numeric_variables]
        y_train = df_train[target_value]
        
        X_test = df_test[all_numeric_variables]
        y_test = df_test[target_value]
        
        method = LogisticRegression(random_state=0)
        fitted_full_model = method.fit(X_train, y_train)
        y_pred = fitted_full_model.predict(X_test)
        
        full_model = accuracy_score(y_test, y_pred)
        result_full_model_etal =  ["The total accuracy using all variable is: " + str(full_model)]
        
        #Analysis Model
        for seg in range(len(seg_data_num)):
            
            mean_value_seg = self.data[seg_data_num[seg]].mean()
            
            # set dataframes of train and test
            
            df_train_seg1 = df_train[df_random_sample[seg_data_num[seg]] >= mean_value_seg]
            df_train_seg2 = df_train[df_random_sample[seg_data_num[seg]] < mean_value_seg]
            df_test_seg1 = df_test[df_random_sample[seg_data_num[seg]] >=  mean_value_seg]
            df_test_seg2 = df_test[df_random_sample[seg_data_num[seg]] < mean_value_seg]
            
            #getting results seg 1 vs seg 1
           
            X_train_seg1 = df_train_seg1[all_numeric_variables]
            y_train_seg1 = df_train_seg1[target_value]
            X_test_seg1 = df_test_seg1[all_numeric_variables]
            y_test_seg1 = df_test_seg1[target_value]
            fitted_model_seg1 = method.fit(X_train_seg1, y_train_seg1)
            y_pred_seg1 = fitted_model_seg1.predict(X_test_seg1)
            y_pred_seg1_fullmodel = fitted_full_model.predict(X_test_seg1)
            
            result_full_model_etal.append("Using: " + seg_data_num[seg] + " Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):"+str(accuracy_score(y_test_seg1, y_pred_seg1)))
            result_full_model_etal.append("Using: " + seg_data_num[seg] + " Model Developed on Full Population (train sample) applied on Seg 1 (test sample):"+str(accuracy_score(y_test_seg1, y_pred_seg1_fullmodel)))
            
            #getting results seg 2 vs seg 2
            
            X_train_seg2 = df_train_seg2[all_numeric_variables]
            y_train_seg2 = df_train_seg2[target_value]
            X_test_seg2 = df_test_seg2[all_numeric_variables]
            y_test_seg2 = df_test_seg2[target_value]
            fitted_model_seg2 = method.fit(X_train_seg2, y_train_seg2)
            y_pred_seg2 = fitted_model_seg2.predict(X_test_seg2)
            y_pred_seg2_fullmodel = fitted_full_model.predict(X_test_seg2)
            result_full_model_etal.append("Using: " + seg_data_num[seg] +" Model Developed on Full Population (train sample) applied on Seg 2 (test sample):" + str(accuracy_score(y_test_seg2, y_pred_seg2_fullmodel)))
            result_full_model_etal.append("Using: " + seg_data_num[seg] +" Model Developed on Seg 2 (train sample) applied on Seg 2 (test sample):" + str(accuracy_score(y_test_seg2, y_pred_seg2))) 
        
        return result_full_model_etal
    
    

# Start 

The user is going to feed the class with the selected file

In [3]:
file = RiskEvaluation('AUTO_LOANS_DATA.csv')

# Basic variables and cleaning

In [4]:
pivot_value = 'ACCOUNT_NUMBER'
target_value = 'BUCKET'
down_payment = 'PROGRAM_NAME'
income_status = 'PROFESSION'
birth_date = 'BIRTH_DATE'
dates_todays = ['REPORTING_DATE','LOAN_OPEN_DATE','EXPECTED_CLOSE_DATE','CUSTOMER_OPEN_DATE']

file.inicial(pivot_value,birth_date,target_value, down_payment,income_status,dates_todays)

,ACCOUNT_NUMBER,CUSTOMER_ID,ORIGINAL_BOOKED_AMOUNT,OUTSTANDING,BUCKET,SEX,BIRTH_DATE,PROFESSION,CAR_TYPE,DOWN_PAYMENT,TYPE,REPORTING_DATE_DAYS_LAPSED,LOAN_OPEN_DATE_DAYS_LAPSED,EXPECTED_CLOSE_DATE_DAYS_LAPSED,CUSTOMER_OPEN_DATE_DAYS_LAPSED
143,144,144,140500.0,0.00,0,M,39,ACTIVE,UNKNOWN,0.5,EMPLOYED,2030,2258,1541,3022
247,248,248,70000.0,0.00,1,F,36,ACTIVE,UNKNOWN,0.5,EMPLOYED,2030,3554,1723,3567
308,309,307,65500.0,0.00,0,M,44,ACTIVE,UNKNOWN,0.5,EMPLOYED,2030,3260,1449,3266
350,351,349,44500.0,0.00,1,M,40,ACTIVE,UNKNOWN,0.5,EMPLOYED,2030,2603,1146,2608
465,466,12,93000.0,0.00,0,UNKNOWN,UNKNOWN,ACTIVE,UNKNOWN,0.0,CORPORATE,2030,3464,2033,7597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900855,36547,35528,90000.0,78956.52,0,M,27,ACTIVE,GELORY,0.5,EMPLOYED,724,1060,-742,1071
900856,39597,38396,92500.0,92826.06,0,F,43,ACTIVE,GELORY,0.5,EMPLOYED,724,725,-1108,738
900857,38016,36905,140250.0,114919.47,0,M,41,ACTIVE,NISSAN,0.0,CORPORATE,724,885,172,936
900858,38899,37739,105000.0,101714.25,0,M,35,ACTIVE,DFSK,0.0,CORPORATE,724,788,-1016,817


# Categorical Results

In [5]:
seg_data_cat =['SEX','PROFESSION','CAR_TYPE','TYPE'] 

In [6]:
file.set_train_cat(target_value,seg_data_cat)

['The total accuracy using all variable is: 0.8888888888888888',
 'Using: SEX Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):0.8870822041553749',
 'Using: SEX Model Developed on Full Population (train sample) applied on Seg 1 (test sample):0.8870822041553749',
 'Using: SEX Model Developed on Full Population (train sample) applied on Seg 2 (test sample):0.8930817610062893',
 'Using: SEX Model Developed on Seg 2 (train sample) applied on Seg 2 (test sample):0.8930817610062893',
 'Using: PROFESSION Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):0.8886059834500318',
 'Using: PROFESSION Model Developed on Full Population (train sample) applied on Seg 1 (test sample):0.8886059834500318',
 'Using: PROFESSION Model Developed on Full Population (train sample) applied on Seg 2 (test sample):0.9230769230769231',
 'Using: PROFESSION Model Developed on Seg 2 (train sample) applied on Seg 2 (test sample):0.9230769230769231',
 'Using: CAR_TYPE Model Dev

# Numerical results

In [7]:
seg_data_num = ['ORIGINAL_BOOKED_AMOUNT','OUTSTANDING','BIRTH_DATE','DOWN_PAYMENT','REPORTING_DATE_DAYS_LAPSED','LOAN_OPEN_DATE_DAYS_LAPSED','EXPECTED_CLOSE_DATE_DAYS_LAPSED','CUSTOMER_OPEN_DATE_DAYS_LAPSED']

In [8]:
file.set_train_num(seg_data_cat,target_value,seg_data_num)

['The total accuracy using all variable is: 0.8906547997457088',
 'Using: ORIGINAL_BOOKED_AMOUNT Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):0.8573667711598746',
 'Using: ORIGINAL_BOOKED_AMOUNT Model Developed on Full Population (train sample) applied on Seg 1 (test sample):0.8573667711598746',
 'Using: ORIGINAL_BOOKED_AMOUNT Model Developed on Full Population (train sample) applied on Seg 2 (test sample):0.9133689839572192',
 'Using: ORIGINAL_BOOKED_AMOUNT Model Developed on Seg 2 (train sample) applied on Seg 2 (test sample):0.9133689839572192',
 'Using: OUTSTANDING Model Developed on Seg 1 (train sample) applied on Seg 1 (test sample):0.8051948051948052',
 'Using: OUTSTANDING Model Developed on Full Population (train sample) applied on Seg 1 (test sample):0.8051948051948052',
 'Using: OUTSTANDING Model Developed on Full Population (train sample) applied on Seg 2 (test sample):0.9243924392439244',
 'Using: OUTSTANDING Model Developed on Seg 2 (train sample)